# 패션 상품 Azure AI Content Understanding 검색 시스템

이 노트북은 Azure AI Content Understanding으로 분석한 패션 상품 정보를 활용한 고도화된 검색 및 분석 시스템을 구현합니다.

## 워크플로우 개요
1. **fashion_products.csv** → 기본 상품 데이터
2. **fashion_content_analyzer.py** → Azure AI Content Understanding으로 분석 및 메타데이터 보강 + Azure AI Search 인덱스에 반영
3. **fashion-sample 인덱스** → AI 분석 필드들 추가 및 정보 업데이트 (이 노트북에서 임베딩을 다시 하지는 않았습니다.)
4. **노트북** → AI 분석 기반 검색 및 필터링 테스트

## 주요 기능
- Azure AI Content Understanding 분석 결과 확인
- 소재/스타일/실루엣별 상품 분포 분석
- AI 분석 기반 상품 검색 및 필터링
- 복합 조건 검색 (색상 + AI 분석 조합)

## 환경 설정 및 라이브러리 import

Azure AI Search와 Content Understanding 분석 데이터를 활용하기 위한 라이브러리들을 import합니다.

In [1]:
import os
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')

# Azure Search 클라이언트
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery, VectorFilterMode
from azure.search.documents.indexes import SearchIndexClient

# 공통
from dotenv import load_dotenv
load_dotenv(override=True)

# 패션 검색 필터 클래스 정의 (AI 분석 포함)
@dataclass
class FashionContentSearchFilter:
    """패션 Content Understanding 검색 필터 클래스"""
    # AI 분석 필드들 (한국어 직접 사용)
    materials: Optional[List[str]] = None        # ai_material
    styles: Optional[List[str]] = None           # ai_style
    occasions: Optional[List[str]] = None        # ai_occasion
    clothing_types: Optional[List[str]] = None   # ai_clothing_type
    primary_colors: Optional[List[str]] = None   # ai_primary_color
    patterns: Optional[List[str]] = None         # ai_pattern
    fits: Optional[List[str]] = None             # ai_fit
    sleeve_lengths: Optional[List[str]] = None   # ai_sleeve_length
    necklines: Optional[List[str]] = None        # ai_neckline
    lengths: Optional[List[str]] = None          # ai_length
    seasons: Optional[List[str]] = None          # ai_season
    formalities: Optional[List[str]] = None      # ai_formality
    target_genders: Optional[List[str]] = None   # ai_target_gender
    target_ages: Optional[List[str]] = None      # ai_target_age
    details: Optional[List[str]] = None          # ai_details
    
    # 기존 필드들 (Computer Vision 등)
    colors: Optional[List[str]] = None           # product_color (Computer Vision)
    brands: Optional[List[str]] = None
    categories: Optional[List[str]] = None
    price_ranges: Optional[List[str]] = None

# Azure 설정
search_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("FASHION_INDEX_NAME", "fashion-sample")
admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

print(f"Search Endpoint: {search_endpoint}")
print(f"Fashion Index Name: {index_name}")
print(f"AI Content Understanding 한국어 직접 검색 모드")
print(f"Environment loaded successfully!")

Search Endpoint: https://hdfn-test-search.search.windows.net
Fashion Index Name: fashion-sample
AI Content Understanding 한국어 직접 검색 모드
Environment loaded successfully!


In [7]:
# Azure Search 클라이언트 초기화
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(admin_key)
)

index_client = SearchIndexClient(
    endpoint=search_endpoint,
    credential=AzureKeyCredential(admin_key)
)

# 인덱스 상태 확인
try:
    index_info = index_client.get_index(index_name)
    print(f"인덱스 '{index_name}' 연결 성공")
    
    # 인덱스 필드 확인 (AI 분석 필드들 포함 여부)
    field_names = [field.name for field in index_info.fields]
    
    ai_fields = [f for f in field_names if f.startswith('ai_')]
    if ai_fields:
        print(f"✅ AI 분석 필드들이 인덱스에 존재합니다: {len(ai_fields)}개")
        print(f"   AI 필드들: {', '.join(ai_fields[:5])}...")
    else:
        print("❌ AI 분석 필드가 없습니다. fashion_content_analyzer.py를 먼저 실행하세요")
    
    # 인덱스 문서 수 확인
    index_results = search_client.search(search_text="*", top=1, include_total_count=True)
    total_count = index_results.get_count()
    print(f"총 문서 수: {total_count:,}개")
    
except Exception as e:
    print(f"인덱스 연결 실패: {e}")
    print("먼저 fashion_index.ipynb를 실행하여 fashion-sample 인덱스를 생성하세요.")

인덱스 'fashion-sample' 연결 성공
✅ AI 분석 필드들이 인덱스에 존재합니다: 20개
   AI 필드들: ai_material, ai_style, ai_clothing_type, ai_primary_color, ai_fit...
총 문서 수: 464개


## AI 분석 결과 현황 파악

Azure AI Content Understanding으로 분석된 패션 정보의 분포를 확인합니다.

In [9]:
def analyze_content_understanding_results():
    """Azure AI Content Understanding 분석 결과 현황 분석"""
    
    print("🤖 Azure AI Content Understanding 분석 결과 현황")
    print("=" * 60)
    
    # AI 분석 필드가 있는 상품들 조회
    results = search_client.search(
        search_text="*",
        filter="ai_material ne null",
        select=["id", "title", "ai_material", "ai_style", "ai_clothing_type", "ai_primary_color", "ai_occasion"],
        top=1000
    )
    
    ai_data = []
    products_with_ai = 0
    
    for result in results:
        if result.get('ai_material'):
            products_with_ai += 1
            ai_data.append({
                'id': result.get('id'),
                'title': result.get('title'),
                'material': result.get('ai_material'),
                'style': result.get('ai_style'),
                'clothing_type': result.get('ai_clothing_type'),
                'primary_color': result.get('ai_primary_color'),
                'occasions': result.get('ai_occasion')
            })
    
    print(f"AI 분석 정보가 있는 상품: {products_with_ai}개")
    if not ai_data:
        print("분석 정보를 가진 상품이 없습니다.")
        print("fashion_content_analyzer.py를 실행하여 이미지를 분석하세요.")
        return None
    
    # 분석 결과 통계
    materials = [item['material'] for item in ai_data if item['material']]
    styles = [item['style'] for item in ai_data if item['style']]
    occasions = [item['occasions'] for item in ai_data if item['occasions']]
    clothing_types = [item['clothing_type'] for item in ai_data if item['clothing_type']]
    primary_colors = [item['primary_color'] for item in ai_data if item['primary_color']]
    
    print(f"\n📊 분석 결과 통계:")
    print(f"   소재 분류: {len(materials)}개")
    print(f"   스타일 분류: {len(styles)}개")
    print(f"   의류 타입: {len(clothing_types)}개")
    print(f"   주요 색상: {len(primary_colors)}개")
    
    print(f"\n🧵 소재 분포 TOP 10:")
    for material, count in Counter(materials).most_common(10):
        print(f"   • {material}: {count}개")
    
    print(f"\n👗 상황별 분포:")
    for occasion, count in Counter(occasions).most_common(10):
        print(f"   • {occasion}: {count}개")

    print(f"\n👗 스타일별 분포:")
    for style, count in Counter(styles).most_common(10):
        print(f"   • {style}: {count}개")

    print(f"\n👕 의류 타입 TOP 10:")
    for clothing_type, count in Counter(clothing_types).most_common(10):
        print(f"   • {clothing_type}: {count}개")
    
    print(f"\n🎨 주요 색상 TOP 10:")
    for color, count in Counter(primary_colors).most_common(10):
        print(f"   • {color}: {count}개")
    
    return {
        'products_with_ai': products_with_ai,
        'ai_data': ai_data,
        'materials': Counter(materials),
        'styles': Counter(styles),
        'clothing_types': Counter(clothing_types),
        'primary_colors': Counter(primary_colors),
        'occasions': Counter(occasions)
    }

ai_analysis = analyze_content_understanding_results();
print("================================")

🤖 Azure AI Content Understanding 분석 결과 현황
AI 분석 정보가 있는 상품: 464개

📊 분석 결과 통계:
   소재 분류: 464개
   스타일 분류: 464개
   의류 타입: 464개
   주요 색상: 464개

🧵 소재 분포 TOP 10:
   • 면: 299개
   • 실크/새틴: 111개
   • 합성섬유: 20개
   • 리넨: 15개
   • 데님: 11개
   • 울/니트: 5개
   • 가죽: 3개

👗 상황별 분포:
   • 알수없음: 464개

👗 스타일별 분포:
   • 캐주얼: 305개
   • 스트리트: 158개
   • 아웃도어: 1개

👕 의류 타입 TOP 10:
   • 티셔츠: 277개
   • 니트/스웨터: 68개
   • 셔츠/블라우스: 55개
   • 원피스: 24개
   • 재킷/블레이저: 24개
   • 슬랙스: 6개
   • 코트: 3개
   • 후드/맨투맨: 3개
   • 바지: 2개
   • 스커트: 2개

🎨 주요 색상 TOP 10:
   • 검정: 109개
   • 초록: 102개
   • 베이지: 94개
   • 회색: 37개
   • 분홍: 33개
   • 네이비: 27개
   • 갈색: 20개
   • 보라: 17개
   • 흰색: 11개
   • 파랑: 8개


## AI 분석 기반 검색 함수 정의

Azure AI Content Understanding 분석 결과를 활용한 고도화된 검색 함수를 구현합니다.

In [ ]:
def fashion_content_search(
    query: str,
    filters: FashionContentSearchFilter = None,
    search_mode: str = "hybrid",
    top: int = 10,
    include_facets: bool = True
):
    """
    AI Content Understanding 분석을 포함한 패션 상품 검색 함수
    
    Args:
        query: 검색어
        filters: AI 분석 포함 패션 필터 객체
        search_mode: 검색 모드 (text/vector/hybrid)
        top: 반환할 결과 수
        include_facets: 패싯 정보 포함 여부
    """
    
    # 기본 검색 파라미터 - AI 분석 필드들 포함
    search_params = {
        "search_text": query,
        "top": top,
        "select": [
            "id", "title", "brand", "category", "keyword", 
            "price_range", "normal_price", "main_text",
            # 기존 색상 필드
            "product_color",
            # AI 분석 필드들
            "ai_material", "ai_style", "ai_occasion", "ai_clothing_type",
            "ai_primary_color", "ai_pattern", "ai_fit",
            "ai_sleeve_length", "ai_neckline", "ai_length", "ai_season",
            "ai_formality", "ai_target_gender", "ai_target_age", "ai_details",
            "ai_caption", "ai_quality_flags",
            # 기존 패션 필드들
            "style", "color", "material", "targetGender", "targetAge", "season"
        ]
    }
    
    # 필터 조건 생성
    filter_conditions = []
    
    if filters:
        # AI 소재 필터
        if filters.materials:
            material_filters = []
            for material in filters.materials:
                material_filters.append(f"ai_material eq '{material}'")
            filter_conditions.append(f"({' or '.join(material_filters)})")
        
        # AI 스타일 필터
        if filters.styles:
            style_filters = []
            for style in filters.styles:
                style_filters.append(f"ai_style eq '{style}'")
            filter_conditions.append(f"({' or '.join(style_filters)})")
        
        # AI 의류 타입 필터
        if filters.clothing_types:
            type_filters = []
            for clothing_type in filters.clothing_types:
                type_filters.append(f"ai_clothing_type eq '{clothing_type}'")
            filter_conditions.append(f"({' or '.join(type_filters)})")
        
        # AI 주요 색상 필터 
        if filters.primary_colors:
            primary_color_filters = []
            for color in filters.primary_colors:
                primary_color_filters.append(f"ai_primary_color eq '{color}'")
            filter_conditions.append(f"({' or '.join(primary_color_filters)})")
        
        # AI 착용 상황 필터 (다중 선택 필드)
        if filters.occasions:
            occasion_filters = []
            for occasion in filters.occasions:
                occasion_filters.append(f"ai_occasion/any(o: o eq '{occasion}')")
            filter_conditions.append(f"({' or '.join(occasion_filters)})")
        
        # AI 패턴 필터 (다중 선택 필드)
        if filters.patterns:
            pattern_filters = []
            for pattern in filters.patterns:
                pattern_filters.append(f"ai_pattern/any(p: p eq '{pattern}')")
            filter_conditions.append(f"({' or '.join(pattern_filters)})")
        
        # AI 핏 필터
        if filters.fits:
            fit_filters = []
            for fit in filters.fits:
                fit_filters.append(f"ai_fit eq '{fit}'")
            filter_conditions.append(f"({' or '.join(fit_filters)})")
        
        # AI 계절 필터 (다중 선택 필드)
        if filters.seasons:
            season_filters = []
            for season in filters.seasons:
                season_filters.append(f"ai_season/any(s: s eq '{season}')")
            filter_conditions.append(f"({' or '.join(season_filters)})")
        
        # AI 포멀도 필터
        if filters.formalities:
            formality_filters = []
            for formality in filters.formalities:
                formality_filters.append(f"ai_formality eq '{formality}'")
            filter_conditions.append(f"({' or '.join(formality_filters)})")
        
        # AI 타겟 성별 필터
        if filters.target_genders:
            gender_filters = []
            for gender in filters.target_genders:
                gender_filters.append(f"ai_target_gender eq '{gender}'")
            filter_conditions.append(f"({' or '.join(gender_filters)})")
        
        # AI 타겟 연령대 필터
        if filters.target_ages:
            age_filters = []
            for age in filters.target_ages:
                age_filters.append(f"ai_target_age eq '{age}'")
            filter_conditions.append(f"({' or '.join(age_filters)})")
        
        # 기존 Computer Vision 색상 필터
        if filters.colors:
            color_filters = []
            for color in filters.colors:
                english_color = normalize_search_term(color, COLOR_MAPPING)
                color_filters.append(f"search.ismatch('{english_color}', 'product_color')")
            filter_conditions.append(f"({' or '.join(color_filters)})")
        
        # 기존 브랜드 필터
        if filters.brands:
            brand_filter = " or ".join([f"brand eq '{brand}'" for brand in filters.brands])
            filter_conditions.append(f"({brand_filter})")
        
        # 기존 가격대 필터
        if filters.price_ranges:
            price_filter = " or ".join([f"price_range eq '{pr}'" for pr in filters.price_ranges])
            filter_conditions.append(f"({price_filter})")
    
    # 필터 조건을 AND로 결합
    if filter_conditions:
        search_params["filter"] = " and ".join(filter_conditions)
    

    if include_facets:
        search_params["facets"] = [
            "brand,count:20",
            "category,count:20", 
            "price_range,count:20",
            # AI 분석 패싯들
            "ai_material,count:15",
            "ai_style,count:10",
            "ai_clothing_type,count:15",
            "ai_primary_color,count:15",
            "ai_fit,count:10",
            "ai_formality,count:8",
            "ai_target_gender,count:5",
            "ai_target_age,count:8"
        ]
    
    # 검색 모드에 따른 추가 설정
    if search_mode == "hybrid":
        search_params["query_type"] = "semantic"
        search_params["semantic_configuration_name"] = "my-semantic-config"
    
    try:
        results = search_client.search(**search_params)
        return [doc for doc in results]
    except Exception as e:
        print(f"검색 중 오류 발생: {e}")
        return []

def display_content_search_results(results, show_facets=True, facets=None):
    """AI 분석 정보를 포함한 검색 결과 출력"""
    
    print(f"\nAI 분석 기반 검색 결과: {len(results)}개 상품 발견\n")
    print("=" * 80)
    
    for i, result in enumerate(results, 1):
        # 기본 정보
        print(f"🏷️  #{i} {result.get('title', 'N/A')}")
        print(f"브랜드: {result.get('brand', 'N/A')}")
        print(f"카테고리: {result.get('category', 'N/A')}")
        print(f"가격대: {result.get('price_range', 'N/A')}")
        
        # AI 분석 정보 (핵심!)
        ai_info = []
        if result.get('ai_material'):
            ai_info.append(f"소재: {result['ai_material']}")
        if result.get('ai_style'):
            ai_info.append(f"스타일: {result['ai_style']}")
        if result.get('ai_clothing_type'):
            ai_info.append(f"타입: {result['ai_clothing_type']}")
        if result.get('ai_fit'):
            ai_info.append(f"핏: {result['ai_fit']}")
        if result.get('ai_formality'):
            ai_info.append(f"포멀도: {result['ai_formality']}")
        
        if ai_info:
            print(f"Content Understanding 분석: {' | '.join(ai_info)}")
        
        # AI 색상 정보
        if result.get('ai_primary_color'):
            print(f"AI Content Understanding 주요색상: {result['ai_primary_color']}")
        
        # Computer Vision 색상 정보
        if result.get('product_color'):
            print(f"Vision 기반 색상: {result['product_color']}")
        
        # AI 캡션
        if result.get('ai_caption'):
            print(f"Content Understanding 설명: {result['ai_caption']}")
        
        # 상품 설명
        if result.get('main_text'):
            print(f"원본 데이터 설명: {result['main_text'][:100]}...")
        
        print("-" * 80)
    
    # 패싯 정보 출력 (AI 분석 포함)
    if show_facets and facets:
        print("\n필터링 옵션:")
        
        ai_facets = ['ai_material', 'ai_style', 'ai_clothing_type', 'ai_primary_color', 'ai_fit']
        for facet_name in ai_facets + ['brand', 'category', 'price_range']:
            if facet_name in facets:
                display_name = facet_name.replace('ai_', 'AI ')
                print(f"\n  {display_name.upper()}:")
                for item in facets[facet_name][:8]:
                    print(f"  • {item['value']} ({item['count']})")
    
    print("\n" + "=" * 80)

## AI 분석 기반 검색 테스트

Azure AI Content Understanding 분석 결과를 활용한 다양한 검색 시나리오를 테스트합니다.

In [5]:
# AI 분석 기반 검색 시나리오 테스트
print("Content Understanding 기반 검색 테스트")
print("="*80)

# 1. 소재별 검색
print("\n1️⃣ 소재별 상품 검색 테스트")
print("-" * 40)

test_materials = ["면", "데님", "울/니트", "가죽"]

for material in test_materials:
    print(f"\n '{material}' 소재 상품 검색:")
    
    material_filter = FashionContentSearchFilter(materials=[material])
    results = fashion_content_search(
        query="*",
        filters=material_filter,
        top=3
    )
    
    if results:
        print(f" {len(results)}개 상품 발견")
        for result in results[:2]:
            ai_material = result.get('ai_material', '정보없음')
            ai_style = result.get('ai_style', '정보없음')
            print(f"    • {result.get('title', 'N/A')[:50]}... | 소재: {ai_material} | 스타일: {ai_style}")
    else:
        print(f"  ❌ {material} 소재 상품을 찾을 수 없습니다")

# 2. 스타일별 검색
print(f"\n\n2️⃣ 스타일별 상품 검색 테스트")
print("-" * 40)

test_styles = ["캐주얼", "비즈니스", "스트리트"]

for style in test_styles:
    print(f"\n'{style}' 스타일 상품 검색:")
    
    style_filter = FashionContentSearchFilter(styles=[style])
    results = fashion_content_search(
        query="*",
        filters=style_filter,
        top=3
    )
    
    if results:
        print(f"  {len(results)}개 상품 발견")
        for result in results[:2]:
            ai_style = result.get('ai_style', '정보없음')
            ai_clothing_type = result.get('ai_clothing_type', '정보없음')
            print(f"    • {result.get('title', 'N/A')[:50]}... | 스타일: {ai_style} | 타입: {ai_clothing_type}")
    else:
        print(f"  ❌ {style} 스타일 상품을 찾을 수 없습니다")

# 3. 한국어 색상 검색 테스트
print(f"\n\n3️⃣ 한국어 색상 검색 테스트")
print("-" * 40)

korean_color_tests = ["검정", "흰색", "베이지", "네이비"]

for color in korean_color_tests:
    print(f"\n'{color}' 색상 상품 검색:")
    
    color_filter = FashionContentSearchFilter(primary_colors=[color])
    results = fashion_content_search(
        query="*",
        filters=color_filter,
        top=3
    )
    
    if results:
        print(f"  {len(results)}개 상품 발견")
        for result in results[:2]:
            ai_color = result.get('ai_primary_color', '정보없음')
            ai_style = result.get('ai_style', '정보없음')
            print(f"    • {result.get('title', 'N/A')[:50]}... | 색상: {ai_color} | 스타일: {ai_style}")
    else:
        print(f"  ❌ {color} 색상 상품을 찾을 수 없습니다")

Content Understanding 기반 검색 테스트

1️⃣ 소재별 상품 검색 테스트
----------------------------------------

 '면' 소재 상품 검색:
 3개 상품 발견
    • [창주스토어 무역센터점][리이/RE RHEE] BLUR LOGO ORGANIC COTTON... | 소재: 면 | 스타일: 스트리트
    • [창주스토어 무역센터점][리이/RE RHEE] RERHEE ORGANIC COTTON T ... | 소재: 면 | 스타일: 스트리트

 '데님' 소재 상품 검색:
 3개 상품 발견
    • [창주스토어 천호점]올세인츠 레미 탑 W093PC-709 ... | 소재: 데님 | 스타일: 캐주얼
    • [창주스토어 판교점]BLR  SUMMER WASHINGBIKER DENIM ZIP-UP V... | 소재: 데님 | 스타일: 스트리트

 '울/니트' 소재 상품 검색:
 3개 상품 발견
    • [창주스토어 무역센터점][Mare di Mari] Flare Sleeveless Knit ... | 소재: 울/니트 | 스타일: 캐주얼
    • [창주스토어 무역센터점][리이/RE RHEE] CUFF SLEEVE PLEATED TOP ... | 소재: 울/니트 | 스타일: 캐주얼

 '가죽' 소재 상품 검색:
 3개 상품 발견
    • [창주스토어 무역센터점][리이/RE RHEE] ASYMMETRIC HEM BOAT NECK... | 소재: 가죽 | 스타일: 캐주얼
    • [창주스토어 판교점][아드베스] Zipped Corset (Black) W22ZCPUP00... | 소재: 가죽 | 스타일: 스트리트


2️⃣ 스타일별 상품 검색 테스트
----------------------------------------

'캐주얼' 스타일 상품 검색:
  3개 상품 발견
    • [창주스토어 무역센터점][Mare di Mari] Stripe Round Neck Top ... | 스타일: 캐주얼 | 타입: 니트/

In [6]:
# 4. 복합 조건 검색: AI 분석 + 색상 조합
print(f"\n4️⃣ 복합 조건 검색 (AI + 색상)")
print("-" * 40)

complex_scenarios = [
    {
        "name": "검정 셔츠",
        "query": "셔츠",
        "filter": FashionContentSearchFilter(
            styles=["캐주얼"],
            primary_colors=["검정"],
            clothing_types=["블라우스/셔츠"]
        )
    },
    {
        "name": "캐주얼 데님",
        "query": "*",
        "filter": FashionContentSearchFilter(
            materials=["데님"],
            styles=["캐주얼"]
        )
    }
]

for scenario in complex_scenarios:
    print(f"\n {scenario['name']}:")
    
    results = fashion_content_search(
        query=scenario['query'],
        filters=scenario['filter'],
        search_mode="hybrid",
        top=5
    )
    
    if results:
        print(f" {len(results)}개 상품 발견")
        display_content_search_results(results[:2], show_facets=False)
    else:
        print(f"  조건에 맞는 상품을 찾을 수 없습니다")


4️⃣ 복합 조건 검색 (AI + 색상)
----------------------------------------

 검정 셔츠:
  조건에 맞는 상품을 찾을 수 없습니다

 캐주얼 데님:
 5개 상품 발견

AI 분석 기반 검색 결과: 2개 상품 발견

🏷️  #1 [창주스토어 천호점]올세인츠 레미 탑 W093PC-709 
브랜드: 올세인츠(백화점)
카테고리: 패션의류
가격대: 20만원대
Content Understanding 분석: 소재: 데님 | 스타일: 캐주얼 | 타입: 셔츠/블라우스 | 핏: 오버사이즈 | 포멀도: 캐주얼
AI Content Understanding 주요색상: 베이지
Vision 기반 색상: White
Content Understanding 설명: 데님 소재의 캐주얼 스타일의 셔츠/블라우스의 베이지 색상
원본 데이터 설명: [창주스토어 천호점]올세인츠 레미 탑 W093PC-709  올세인츠(백화점) 패션의류 올세인츠의 레미 탑은 현대적인 감각과 세련된 디자인을 갖춘 제품으로, 다양한 스타일링에 적합합...
--------------------------------------------------------------------------------
🏷️  #2 [창주스토어 무역센터점][리이/RE RHEE] ELASTICATED HEM SLEEVELESS SHIRT TOP 
브랜드: 리이
카테고리: 패션의류
가격대: 10만원대
Content Understanding 분석: 소재: 데님 | 스타일: 캐주얼 | 타입: 원피스 | 핏: 슬림 | 포멀도: 캐주얼
AI Content Understanding 주요색상: 베이지
Vision 기반 색상: Grey White
Content Understanding 설명: 데님 소재의 캐주얼 스타일의 원피스의 베이지 색상
원본 데이터 설명: [창주스토어 무역센터점][리이/RE RHEE] ELASTICATED HEM SLEEVELESS SHIRT TOP  리이 패션의류 허리 밴딩이 포함된 팬츠 및 쇼츠 

## AI 분석 품질 및 커버리지 평가

Content Understanding 분석 결과의 품질과 데이터 커버리지를 평가합니다.

In [9]:
def analyze_ai_analysis_quality():
    """AI 분석 품질 및 커버리지 평가"""
    
    print("AI Content Understanding 분석 품질 평가")
    print("=" * 60)
    
    # 전체 상품 수 조회
    all_results = search_client.search(
        search_text="*",
        select=["id", "title", "ai_material", "ai_style", "ai_clothing_type", 
                "ai_primary_color", "ai_caption", "product_color"],
        top=1000
    )
    
    total_products = 0
    ai_analysis_stats = {
        'has_material': 0,
        'has_style': 0,
        'has_clothing_type': 0,
        'has_primary_color': 0,
        'has_caption': 0,
        'has_complete_analysis': 0,
        'has_both_color_analysis': 0  # AI + Computer Vision 색상 모두 있음
    }
    
    for result in all_results:
        total_products += 1
        
        # AI 분석 필드별 커버리지 확인
        if result.get('ai_material'):
            ai_analysis_stats['has_material'] += 1
        if result.get('ai_style'):
            ai_analysis_stats['has_style'] += 1
        if result.get('ai_clothing_type'):
            ai_analysis_stats['has_clothing_type'] += 1
        if result.get('ai_primary_color'):
            ai_analysis_stats['has_primary_color'] += 1
        if result.get('ai_caption'):
            ai_analysis_stats['has_caption'] += 1
        
        # 완전한 분석 (주요 필드 4개 모두 있음)
        if all([result.get('ai_material'), result.get('ai_style'), 
                result.get('ai_clothing_type'), result.get('ai_primary_color')]):
            ai_analysis_stats['has_complete_analysis'] += 1
        
        # AI + Computer Vision 색상 분석 모두 있음
        if result.get('ai_primary_color') and result.get('product_color'):
            ai_analysis_stats['has_both_color_analysis'] += 1
    
    # 분석 결과 출력
    print(f"\nAI 분석 커버리지:")
    print(f"   전체 상품: {total_products:,}개")
    
    for field, count in ai_analysis_stats.items():
        coverage_rate = (count / total_products * 100) if total_products > 0 else 0
        field_display = field.replace('has_', '').replace('_', ' ').title()
        print(f"   {field_display}: {count:,}개 ({coverage_rate:.1f}%)")
    
    # 샘플 AI 분석 결과 표시
    print(f"\nAI 분석 결과 샘플:")
    sample_results = search_client.search(
        search_text="*",
        filter="ai_material ne null and ai_caption ne null",
        select=["title", "ai_material", "ai_style", "ai_clothing_type", "ai_caption"],
        top=5
    )
    
    for i, result in enumerate(sample_results, 1):
        title = result.get('title', 'N/A')[:40]
        material = result.get('ai_material', 'N/A')
        style = result.get('ai_style', 'N/A')
        clothing_type = result.get('ai_clothing_type', 'N/A')
        caption = result.get('ai_caption', 'N/A')
        
        print(f"   {i}. {title}...")
        print(f"      {material} | {style} | {clothing_type}")
        print(f"      {caption}")
        print()

# AI 분석 품질 평가 실행
analyze_ai_analysis_quality()

AI Content Understanding 분석 품질 평가

AI 분석 커버리지:
   전체 상품: 464개
   Material: 464개 (100.0%)
   Style: 464개 (100.0%)
   Clothing Type: 464개 (100.0%)
   Primary Color: 464개 (100.0%)
   Caption: 464개 (100.0%)
   Complete Analysis: 464개 (100.0%)
   Both Color Analysis: 462개 (99.6%)

AI 분석 결과 샘플:
   1. [창주스토어 더현대대구][CC콜렉트] C252PST051 스트라이프 스퀘...
      실크/새틴 | 스트리트 | 니트/스웨터
      실크/새틴 소재의 스트리트 스타일의 니트/스웨터의 베이지 색상

   2. [창주스토어 무역센터점][리이/RE RHEE] BLUR LOGO ORGA...
      면 | 스트리트 | 티셔츠
      면 소재의 스트리트 스타일의 티셔츠의 초록 색상

   3. [창주스토어 무역센터점][Mare di Mari] Stripe Round...
      실크/새틴 | 캐주얼 | 니트/스웨터
      실크/새틴 소재의 캐주얼 스타일의 니트/스웨터의 네이비 색상

   4. [창주스토어 무역센터점][리이/RE RHEE] HALTER NECK KN...
      실크/새틴 | 스트리트 | 니트/스웨터
      실크/새틴 소재의 스트리트 스타일의 니트/스웨터의 검정 색상

   5. [창주스토어 무역센터점][리이/RE RHEE] RERHEE ORGANIC...
      면 | 스트리트 | 티셔츠
      면 소재의 스트리트 스타일의 티셔츠의 검정 색상

